In [1]:
import pandas as pd

df= pd.read_csv("Global_Space_Exploration_Dataset.csv")

df.head()

,Country,Year,Mission Name,Mission Type,Launch Site,Satellite Type,Budget (in Billion $),Success Rate (%),Technology Used,Environmental Impact,Collaborating Countries,Duration (in Days)
0,China,2008,Sharable tertiary superstructure,Manned,Sheilatown,Communication,16.20,90,Nuclear Propulsion,Medium,"France, UK, Russia",112
1,Japan,2018,Re-engineered composite flexibility,Manned,New Ericfurt,Communication,29.04,99,Solar Propulsion,High,"Germany, Israel",236
2,Israel,2013,Reactive disintermediate projection,Manned,Port Kaitlynstad,Communication,28.73,54,AI Navigation,Medium,"China, Israel, USA",238
3,UAE,2010,Grass-roots 6thgeneration implementation,Unmanned,Mariastad,Spy,37.27,58,Traditional Rocket,Low,USA,186
4,India,2006,Balanced discrete orchestration,Manned,North Jasonborough,Weather,18.95,91,Solar Propulsion,Medium,"Israel, China, India",277


In [2]:
df.drop_duplicates(inplace=True)

In [3]:
df.columns=df.columns.str.replace(" ","_").str.replace("(","").str.replace(")","")


In [4]:
df['Success_Rate_%']=df['Success_Rate_%'].astype(float)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Country                  3000 non-null   object 
 1   Year                     3000 non-null   int64  
 2   Mission_Name             3000 non-null   object 
 3   Mission_Type             3000 non-null   object 
 4   Launch_Site              3000 non-null   object 
 5   Satellite_Type           3000 non-null   object 
 6   Budget_in_Billion_$      3000 non-null   float64
 7   Success_Rate_%           3000 non-null   float64
 8   Technology_Used          3000 non-null   object 
 9   Environmental_Impact     3000 non-null   object 
 10  Collaborating_Countries  3000 non-null   object 
 11  Duration_in_Days         3000 non-null   int64  
dtypes: float64(2), int64(2), object(8)
memory usage: 281.4+ KB


**Read Data from Staging Table**

In [8]:
from sqlalchemy import create_engine

server = '.'
database='SpaceDW'
engine = create_engine(f"mssql+pyodbc://@{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes")

df.to_sql('Staging_SpaceMissions',con=engine,if_exists='replace',index=False)

42

**Preview Sample Data from Staging Table**

In [12]:
from sqlalchemy import text 

with engine.connect() as conn:
    result = conn.execute(text("SELECT TOP 5 * FROM Staging_SpaceMissions"))
    for row in result:
        print(row)


('China', 2008, 'Sharable tertiary superstructure', 'Manned', 'Sheilatown', 'Communication', 16.2, 90.0, 'Nuclear Propulsion', 'Medium', 'France, UK, Russia', 112)
('Japan', 2018, 'Re-engineered composite flexibility', 'Manned', 'New Ericfurt', 'Communication', 29.04, 99.0, 'Solar Propulsion', 'High', 'Germany, Israel', 236)
('Israel', 2013, 'Reactive disintermediate projection', 'Manned', 'Port Kaitlynstad', 'Communication', 28.73, 54.0, 'AI Navigation', 'Medium', 'China, Israel, USA', 238)
('UAE', 2010, 'Grass-roots 6thgeneration implementation', 'Unmanned', 'Mariastad', 'Spy', 37.27, 58.0, 'Traditional Rocket', 'Low', 'USA', 186)
('India', 2006, 'Balanced discrete orchestration', 'Manned', 'North Jasonborough', 'Weather', 18.95, 91.0, 'Solar Propulsion', 'Medium', 'Israel, China, India', 277)


**Load Distinct Countries into DimCountry**

In [14]:
with engine.connect() as conn:
    df_country=pd.read_sql(text("""
        SELECT DISTINCT Country
        FROM Staging_SpaceMissions
        WHERE Country IS NOT NULL
     """),conn)

df_country.to_sql('DimCountry', con=engine,if_exists='append',index=False)    

10

**Load Dimensions Tables**

In [19]:
with engine.connect() as conn:
    df_mission_type=pd.read_sql(text("""
        SELECT DISTINCT Mission_Type
        FROM Staging_SpaceMissions
        WHERE Mission_Type IS NOT NULL
         """),conn)

df_mission_type.columns=['MissionType']
df_mission_type.to_sql('DimMissionType',con=engine, if_exists='append',index=False)

2

In [89]:
with engine.connect() as conn:
    df_technology=pd.read_sql(text("""
        SELECT DISTINCT Technology_Used
        FROM Staging_SpaceMissions
        WHERE Technology_Used IS NOT NULL
         """),conn)

df_technology.columns=['Technology']
df_technology.to_sql('DimTechnology',con=engine, if_exists='append',index=False)

5

In [90]:
with engine.connect() as conn:
    df_collab_country = pd.read_sql(text("""
        SELECT DISTINCT Collaborating_Countries
        FROM Staging_SpaceMissions
        WHERE Collaborating_Countries IS NOT NULL
    """), conn)

df_collab_country.columns = ['CollaboratingCountries']

df_collab_country.to_sql('DimCollaboratingCountries', con=engine, if_exists='append', index=False)


639

In [30]:
with engine.connect() as conn:
    df_years=pd.read_sql(text("""
        SELECT DISTINCT Year , (Year / 10)*10               
        FROM Staging_SpaceMissions
        WHERE Year IS NOT NULL
        """),conn)
df_years.columns=['Year_Key','Decade']
df_years.to_sql('DimYear',con=engine,if_exists='append',index=False)


26

In [ ]:
dimensions={
    'DimLaunchSite':'Launch_Site',
    'DimSatelliteType':'Satellite_Type',
    'DimEnvironmentalImpact':'Environmental_Impact'
}

for dim_table, col in dimensions.items():
    with engine.connect() as conn:
        df_dim = pd.read_sql(text(f"""
            SELECT DISTINCT [{col}]
            FROM Staging_SpaceMissions
            WHERE [{col}] IS NOT NULL
        """), conn)

    df_dim.columns=[col.replace('_','')]

    df_dim.to_sql(dim_table,con=engine,if_exists='append',index=False)    


**Load All Dimension Tables for Joining**

In [91]:
df_staging = pd.read_sql("SELECT * FROM Staging_SpaceMissions", engine)

df_country = pd.read_sql("SELECT Country_Key, Country FROM DimCountry", engine)
df_launch = pd.read_sql("SELECT LaunchSite_Key, LaunchSite FROM DimLaunchSite", engine)
df_year = pd.read_sql("SELECT Year_Key, Decade FROM DimYear", engine)
df_mission_type = pd.read_sql("SELECT MissionType_Key, MissionType FROM DimMissionType", engine)
df_satellite_type = pd.read_sql("SELECT SatelliteType_Key, SatelliteType FROM DimSatelliteType", engine)
df_technology = pd.read_sql("SELECT Tech_Key, Technology FROM DimTechnology", engine)
df_envImpact = pd.read_sql("SELECT EnvImpact_Key, EnvironmentalImpact FROM DimEnvironmentalImpact", engine)
df_collab_country = pd.read_sql("SELECT CollabCountries_Key, CollaboratingCountries FROM DimCollaboratingCountries", engine)


**Merge Staging Table with Dimension Tables**

In [92]:
df = df_staging.merge(df_country, on='Country')
print("After Country Merge:", df.shape)
df = df.merge(df_year, left_on='Year', right_on='Decade')
print("After Year Merge:", df.shape)
df = df.merge(df_launch, left_on='Launch_Site', right_on='LaunchSite')
print("After Launch Site Merge:", df.shape)
df = df.merge(df_envImpact, left_on='Environmental_Impact', right_on='EnvironmentalImpact')
print("After Environmental Impact Merge:", df.shape)
df = df.merge(df_mission_type, left_on='Mission_Type', right_on='MissionType')
print("After Mission Type Merge:", df.shape)
df = df.merge(df_satellite_type, left_on='Satellite_Type', right_on='SatelliteType')
print("After Satellite Type Merge:", df.shape)
df = df.merge(df_technology, left_on='Technology_Used', right_on='Technology')
print("After Technology Merge:", df.shape)
df = df.merge(df_collab_country, left_on='Collaborating_Countries', right_on='CollaboratingCountries')
print("After Collaborating Countries Merge:", df.shape)


After Country Merge: (3000, 13)
After Year Merge: (2926, 15)
After Launch Site Merge: (5852, 17)
After Environmental Impact Merge: (11704, 19)
After Mission Type Merge: (11704, 21)
After Satellite Type Merge: (23408, 23)
After Technology Merge: (46816, 25)
After Collaborating Countries Merge: (93632, 27)


In [86]:
df.columns

Index(['Country', 'Year', 'Mission_Name', 'Mission_Type', 'Launch_Site',
       'Satellite_Type', 'Budget_in_Billion_$', 'Success_Rate_%',
       'Technology_Used', 'Environmental_Impact', 'Collaborating_Countries',
       'Duration_in_Days', 'Country_Key', 'Year_Key', 'Decade',
       'LaunchSite_Key', 'LaunchSite', 'EnvImpact_Key', 'EnvironmentalImpact',
       'MissionType_Key', 'MissionType', 'SatelliteType_Key', 'SatelliteType',
       'Tech_Key', 'Technology', 'CollabCountries_Key',
       'CollaboratingCountries'],
      dtype='object')

**Create and Load the Fact Table**

In [95]:
fact_df = df[[  
    'Mission_Name', 
    'Country_Key', 
    'Year_Key', 
    'LaunchSite_Key',
    'MissionType_Key',
    'SatelliteType_Key',
    'Tech_Key',
    'EnvImpact_Key',
    'CollabCountries_Key',
    'Budget_in_Billion_$',
    'Success_Rate_%',
    'Duration_in_Days'
]].copy()

fact_df.rename(columns={
    'Mission_Name': 'MissionName',
    'Budget_in_Billion_$': 'Budget',
    'Success_Rate_%': 'SuccessRate',
    'Duration_in_Days': 'Duration'
}, inplace=True)



fact_df.to_sql('FactMissions', con=engine, if_exists='append', index=False)

20